In [ ]:
%pip install -q -U transformers peft accelerate optimum
%pip install bitsandbytes
%pip install auto-gptq
%pip install nltk
%pip install trl
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Generazione risposta singola


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

# Caricamento del modello base

base_model_id = "TechxGenus/gemma-7b-GPTQ"
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
model = AutoModelForCausalLM.from_pretrained(base_model_id,quantization_config=quantization_config_loading, device_map="auto", trust_remote_code=True)
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

# Definizione del prompt e della frase di input
question = "Recognition of proper nouns in Japanese text has been studied as a part of the more general problem of morphological analysis in Japanese text processing -LRB- -LSB- 1 -RSB- -LSB- 2 -RSB- -RRB- "
eval_prompt = f"""Extract the entities for the following labels from the given text and provide the results in JSON format
- Entities must be extracted exactly as mentioned in the text.
- Return each entity under its label without creating new labels.
- Provide a list of entities for each label, ensuring that if no entities are found for a label, an empty list is returned.
- Accuracy and relevance in your responses are key.

Lables and their Descriptions:
- Task: applications, problems to solve, systems to construct.
- Method: methods, models, systems to use, tools, components of a system.
- Metric: metrics, measures, or entities that can express quality of a system/method.
- Material: data, datasets, resources, Corpus, Knowledge base.
- OtherScientificTerm: phrases that are a scientific terms but do not fall into any of the above classes.
- Generic: general terms or pronouns that may refer to a entity but are not themselves informative.

### Input text: {question}

### Response:"""

# Caricamento del modello con il fine-tuning tramite PEFT

from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "Andro9669/gemma-7b-ner")

# Generazione della risposta
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")
ft_model.eval()
with torch.no_grad():
  result = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens= 150,do_sample = False)[0], skip_special_tokens=True)
  print(result)

Generazione risposte con test set di SCIERC e risultati in file JSON

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

# Caricamento del modello base
base_model_id = "TechxGenus/gemma-7b-GPTQ"
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
model = AutoModelForCausalLM.from_pretrained(base_model_id,quantization_config=quantization_config_loading, device_map="auto", trust_remote_code=True)
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

# Caricamento del modello fine-tuned
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "Andor9669/gemma-7b-ner")
path = "/content/drive/MyDrive/Tesi/result/result_gemma7b_no_inc_sftt.json"

import json

# Per ogni riga del dataset si effettua la predizione e si scrive il risultato nel file di output
with open("/content/drive/MyDrive/Tesi/dataset/scierc_test_no_inc.json") as f:
    for line in f:
        #convert the line in a json object
        data = json.loads(line)
        question = data["input"]
        eval_prompt = f"""Extract the entities for the following labels from the given text and provide the results in JSON format
- Entities must be extracted exactly as mentioned in the text.
- Return each entity under its label without creating new labels.
- Provide a list of entities for each label, ensuring that if no entities are found for a label, an empty list is returned.
- Accuracy and relevance in your responses are key.

Lables and their Descriptions:
- Task: applications, problems to solve, systems to construct.
- Method: methods, models, systems to use, tools, components of a system.
- Metric: metrics, measures, or entities that can express quality of a system/method.
- Material: data, datasets, resources, Corpus, Knowledge base.
- OtherScientificTerm: phrases that are a scientific terms but do not fall into any of the above classes.
- Generic: general terms or pronouns that may refer to a entity but are not themselves informative.

### Input text: {question}

### Response:"""
        model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")
        ft_model.eval()
        result = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens= 150,do_sample = False)[0], skip_special_tokens=True)
        with open(path, 'a') as outfile:
          json_object = {"prediction": result}
          json.dump(json_object, outfile)
          outfile.write('\n')
    